In [1]:
import pandas as pd
import numpy as np
import time
import re

In [2]:
# gp = pd.read_csv('../gameprograms230920.csv')
# gp['usage'] = '게임'

In [3]:
today = time.strftime('%y%m%d')
# pd.DataFrame(gp).to_csv(f'../gameprograms{today}.csv', index=None)

In [4]:
gr_origin = pd.read_csv('../gamerequirements_230922.csv')

In [5]:
gr_origin

,name,cpu,ram,gpu,dx,os,sto,spec_class
0,A Bird Story,> Intel Pentium III 800 MHz,2 GB RAM,1024x768 High Color +,Version 9.0c,"XP, Vista, 7, 8",0.2,0
1,A Blind Legend,Dual Core CPU,1 GB RAM,generally everything made since 2004 should work.,Version 9.0,Windows XP SP2+,0.4,0
2,A Boy and His Blob,"AMD Athlon 64 X2 Dual Core Processor 4600+, 2....",1 GB RAM,NVIDIA GeForce 9600 GT 512 MB / ATI Radeon HD ...,NaN,Windows 7-32Bit or Windows 8-32Bit,3,0
3,A Boy and His Blob,AMD Athlon 7850 Dual Core Processor 2.8GHz / I...,2 GB RAM,NVIDIA GeForce 9600 GT 1GB / ATI Radeon HD 465...,NaN,Windows 7-64Bit or Windows 8-64Bit,3,1
4,A City Sleeps,Intel Core2 Duo 3.00GHz,2 GB RAM,ATI Radeon 2400,9.0c,Windows 7 / 8,NaN,0
...,...,...,...,...,...,...,...,...
12090,99Vidas,Intel Core i5 @ 2.66GHz / AMD Phenom,4 GB RAM,NVIDIA GeForce GTX 460 (1GB) / ATI Radeon HD 5...,Version 11,Microsoft Windows 7 SP2 / Windows 8.1 / Window...,2,1
12091,9th Company: Roots of Terror,Intel Pentium 4 2400 MHz and higher,2048MB,3D accelerator with 128Mb video memory (NVIDIA...,DirectX 9.0 or higher,Windows XP SP2/Vista,NaN,0
12092,Ész-kerék,NaN,NaN,NaN,NaN,Dos,NaN,0
12093,été,NaN,NaN,NaN,NaN,Windows 7,4,0


In [6]:
## 용량 시작.........................
gr_origin['sto'].str.contains('4,500')

0        False
1        False
2        False
3        False
4          NaN
         ...  
12090    False
12091      NaN
12092      NaN
12093    False
12094    False
Name: sto, Length: 12095, dtype: object

In [7]:
gr_origin.loc[gr_origin['sto'].str.contains('3 meg of Hard Drive Space \(for shareware\)', na=False), 'sto']

Series([], Name: sto, dtype: object)

In [8]:
sto_regex_map = {',500': '500', '(a|A)t least ': '', '(M|m)b': 'MB', 'from': '', 'meg': 'MB',
                 'Hard Drive Space: ': '', 'Approximately ': '', '7200 RPM or SSD drive, ': '',
                 'Solid State Drive \(SSD\) ': '', 'Minimum ': '', '(a|A)bout ': '', 'Media: ': '',
                 'Hard disk with ': '', 'DirectX® 9\.0c': '', 'Up to ': '', '7200rpm or faster': '',
                 'Complete installation:': '', 'Installation requires ': '', 'approx\. ': '',
                 'Hard Drive Space: ': '', 'Hard disk drive with ': '', 'To install \~ ': '',
                 'SSD strongly recommended': '', 'Hard disk space ': '', 'We do not know the final size of our game yet': '',
                 'High Performance SSD Recommended with ': '', 'Please reserve ': '', 'SSD with ': '',
                 }
after_map = {r'(GB|Gb|[a-z]).*': ''}

In [9]:
sto = gr_origin.sto.replace(regex=sto_regex_map).replace(regex=after_map)

In [10]:
sto_list = []
for i in range(len(sto)):
    if pd.isnull(sto.loc[i]):
        sto_list.append(None)
        continue
    sto.loc[i] = sto.loc[i].replace(',', '.')
    mbflag = 'MB' in sto.loc[i]
    sto.loc[i] = re.sub('MB.*', '', sto.loc[i])
    sto.loc[i] = re.sub('[^\d.]', '', sto.loc[i])
    if mbflag: sto.loc[i] = str(float(sto.loc[i]) / 1000)
    if sto.loc[i] == '': sto.loc[i] = None
    sto_list.append(sto.loc[i])
    percent = i / len(gr_origin.sto) * 100
    print(f'\rgr: {i+1}/{len(gr_origin.sto)} ({percent:.0f}%)', end='')

gr: 12095/12095 (100%)

In [11]:
# for a, b in zip(gr_origin.sto, sto_list):
#     print(f'{a}::{b}')

In [12]:
# gr_origin.sto = sto_list

In [24]:
## 램 시작............
# gr_origin.ram.unique()

In [14]:
ram_regex_map = {r'(M|m)b': 'MB'}
after_map = {r'(GB|Gb|[a-z]).*': ''}

In [15]:
ram = gr_origin.ram.replace(regex=ram_regex_map).replace(regex=after_map)

In [48]:
k = gr_origin.loc[gr_origin.ram.str.contains(r'^Memory', na=False), 'ram']
k

9917    Memory 1 GB
9918    Memory 2 GB
Name: ram, dtype: object

In [49]:
# n = 0

In [52]:
# idx = k.keys()[n]
# n += 1
# gr_origin.loc[idx, 'ram']

'Memory 2 GB'

In [53]:
# gr_origin.loc[idx, 'ram'] = '2 GB'

In [54]:
# pd.DataFrame(gr_origin).to_csv(f'../gamerequirements{today}.csv', index=None)

In [58]:
ram_list = []
for i in range(len(ram)):
    if pd.isnull(ram.loc[i]):
        ram_list.append(None)
        continue
    ram.loc[i] = ram.loc[i].replace(',', '.')
    mbflag = 'MB' in ram.loc[i]
    ram.loc[i] = re.sub('MB.*', '', ram.loc[i])
    ram.loc[i] = re.sub('[^\d\.]', '', ram.loc[i])
    if mbflag: ram.loc[i] = str(float(ram.loc[i]) / 1024)
    if ram.loc[i] == '': ram.loc[i] = None
    ram_list.append(ram.loc[i])
    percent = i / len(gr_origin.ram) * 100
    print(f'\rgr: {i+1}/{len(gr_origin.ram)} ({percent:.0f}%)', end='')

gr: 12095/12095 (100%)

In [56]:
# pd.DataFrame(gr_origin).to_csv(f'../gamerequirements_{today}.csv', index=None)

In [60]:
# for a, b in zip(gr_origin.ram, ram_list):
#     print(f'{a}::{b}')

In [70]:
gr_origin.cpu

0                              > Intel Pentium III 800 MHz
1                                            Dual Core CPU
2        AMD Athlon 64 X2 Dual Core Processor 4600+, 2....
3        AMD Athlon 7850 Dual Core Processor 2.8GHz / I...
4                                  Intel Core2 Duo 3.00GHz
                               ...                        
12090                 Intel Core i5 @ 2.66GHz / AMD Phenom
12091                  Intel Pentium 4 2400 MHz and higher
12092                                                  NaN
12093                                                  NaN
12094                                              1.7 Ghz
Name: cpu, Length: 12095, dtype: object

In [325]:
## CPU 시작.................
cfile = pd.read_csv('../part_cpu_wb230921.csv')
cpu = cfile.name
cpu_mc = cfile.memory_clock
cpu_bench = cfile.bench_mark
req_cpu = gr_origin.cpu

In [277]:
cpu

0         AMD 라이젠5-5세대 7600
1      AMD 라이젠7-5세대 7800X3D
2       인텔 코어i5-13세대 13400F
3        AMD 라이젠5-5세대 7500F
4        AMD 라이젠5-5세대 7600X
               ...         
420             인텔 펜티엄 G870
421             인텔 펜티엄 G850
422             인텔 펜티엄 G840
423            인텔 펜티엄 E2200
424          인텔 코어2듀오 E6600
Name: name, Length: 425, dtype: object

In [278]:
req_cpu

0                              > Intel Pentium III 800 MHz
1                                            Dual Core CPU
2        AMD Athlon 64 X2 Dual Core Processor 4600+, 2....
3        AMD Athlon 7850 Dual Core Processor 2.8GHz / I...
4                                  Intel Core2 Duo 3.00GHz
                               ...                        
12090                 Intel Core i5 @ 2.66GHz / AMD Phenom
12091                  Intel Pentium 4 2400 MHz and higher
12092                                                  NaN
12093                                                  NaN
12094                                              1.7 Ghz
Name: cpu, Length: 12095, dtype: object

In [322]:
cpu.loc[cpu.str.contains('애슬론', na=False)]

150       AMD 애슬론 3000G
241       AMD 애슬론 200GE
347    AMD 애슬론II-X4 620
370    AMD 애슬론II-X3 425
405    AMD 애슬론II-X4 630
Name: name, dtype: object

In [312]:
req_cpu.loc[req_cpu.str.contains('II', na=False)]

0                              > Intel Pentium III 800 MHz
20       Intel Core i3-2100 (2 * 3100) or equivalent / ...
24                                  Pentium II  350Mhz\r\n
56                 Intel Pentium III or AMD Athlon 800 MHz
62                                            P III 600MHz
                               ...                        
12014                                        PIII 700 MHz 
12020      Pentium III 800MHz or 100% compatible processor
12028                                Pentium II 233MHz CPU
12029                               Pentium III 500MHz CPU
12037     Intel Pentium III 800 MHz or AMD Athlon 800 MHz 
Name: cpu, Length: 977, dtype: object

In [281]:
gr_origin.loc[5492]

name                                            LEGO The Hobbit
cpu                AMD or Intel Quad Core running at 4*2600 Mhz
ram                                                    4 GB RAM
gpu           NVIDIA GeForce GTX 480 or ATI Radeon HD 5850 o...
dx                                                   Version 11
os                                         Windows XP/Vista/7/8
sto                                                          10
spec_class                                                    1
Name: 5492, dtype: object

In [291]:
re.sub('\([^)]+\)', '', '(adasd)')

''

In [313]:
cpu_regex_map = {'인텔': 'Intel ', '라이젠': 'Ryzen ', '코어2': 'Core2 ', '코어': 'Core ', '스레드리퍼': 'Threadripper ',
          r'-\d+세대': ' ', '펜티엄': 'Pentium ', '셀러론': 'Celeron ', '골드': 'Gold ',
          'X-시리즈': ' ', '어벤져스 에디션': ' ', '제온': 'Xeon ', '브론즈': 'Beonze ',
          '스케일러블': ' ', '실버': 'Silver ', '애슬론': 'Athlon', '듀오': ' Duo', '쿼드 ': ' Quad ',
          '플래티넘': 'Platinum ', '\+ ': ' ', 'V6': ' v6 ', 'V5': ' v5 ', 'V2': ' v2 ', 'V4': ' v4 ',
                'V3': ' v3 '}
req_regex_map = {'@': ' ', '\+': '', r' ?(G|g)(H|h)(Z|z)': 'GHz', r' ?(M|m)(H|h)(Z|z)': 'MHz ', ' ?(C|c)ore': ' Core',
                 'v2': ' v2', 'quad': 'Quad', 'duo': 'Duo', '\n': ' ', '\r': ' ', ',': '.', '(>|<)': '', ' GMHz': 'GHz',
                 '\([^)]+\)': '', 'P266': '266', '4\*2600': '10400', 'Pentium': ' Pentium ', 'II+': ''}
none_regex_map = {r' +': ' ', '\*': ''}

In [321]:
cpu_reg = cpu.replace(regex=cpu_regex_map).replace(regex=none_regex_map)
req_reg = req_cpu.replace(regex=req_regex_map).replace(regex=none_regex_map)

In [315]:
c_names = cpu_reg.str.split(r' |-')
reqc_names = req_reg.str.split(r' |-|/')

In [316]:
cmc = cpu_mc.mul(0.001)

for cn, cm in zip(c_names, cmc):
    if pd.isnull(cm): cn.append(None)
    else: cn.append(f'{cm}GHz')

In [317]:
for lst in reqc_names:
    if type(lst) == float: continue
    for i in range(len(lst)):
        if 'MHz' in lst[i]:
            try:
                val = str(float(lst[i].replace('MHz', '')) * 0.001)
                lst[i] = f'{val}GHz'
            except:
                print(lst)

In [318]:
c_names

0             [AMD, Ryzen, 5, 7600, 5.2GHz]
1          [AMD, Ryzen, 7, 7800X3D, 5.2GHz]
2         [Intel, Core, i5, 13400F, 4.8GHz]
3            [AMD, Ryzen, 5, 7500F, 5.2GHz]
4            [AMD, Ryzen, 5, 7600X, 5.2GHz]
                       ...                 
420            [Intel, Pentium, G870, None]
421            [Intel, Pentium, G850, None]
422            [Intel, Pentium, G840, None]
423         [Intel, Pentium, E2200, 0.8GHz]
424    [Intel, Core2, Duo, E6600, 1.066GHz]
Name: name, Length: 425, dtype: object

In [319]:
reqc_names

0                             [, Intel, Pentium, 0.8GHz, ]
1                                        [Dual, Core, CPU]
2        [AMD, Athlon, 64, X2, Dual, Core, Processor, 4...
3        [AMD, Athlon, 7850, Dual, Core, Processor, 2.8...
4                             [Intel, Core2, Duo, 3.00GHz]
                               ...                        
12090          [Intel, Core, i5, 2.66GHz, , , AMD, Phenom]
12091             [Intel, Pentium, 4, 2.4GHz, and, higher]
12092                                                  NaN
12093                                                  NaN
12094                                             [1.7GHz]
Name: cpu, Length: 12095, dtype: object

In [328]:
np.inf < 90

False

In [331]:
np.inf < cpu_bench.loc[1]

False

In [332]:
id_list = [None] * len(reqc_names)
for i in range(len(reqc_names)):
    rn = reqc_names.loc[i]
    if type(rn) == float:
        continue
    max_val = 0
    min_bench = np.inf
    for j in range(len(c_names)):
        cn = c_names.loc[j]
        res = len(set(rn) & set(cn))
        val = res / float(len(set(rn) | set(cn))) * 100
        if val >= max_val and min_bench > cpu_bench.loc[j]:
            max_val = val
            min_bench = cpu_bench.loc[j]
            id_list[i] = f'{j+1}'
    percent = i / len(reqc_names) * 100
    print(f'\rcpu: {i+1}/{len(reqc_names)} ({percent:.0f}%)', end='')

cpu: 12095/12095 (100%)

In [ ]:
## 1. 게임 사양 <-> 벤치 이름 매칭
## 2. 벤치 값 이상이면서 가장 작은 부품 매칭

In [341]:
cpu

0         AMD 라이젠5-5세대 7600
1      AMD 라이젠7-5세대 7800X3D
2       인텔 코어i5-13세대 13400F
3        AMD 라이젠5-5세대 7500F
4        AMD 라이젠5-5세대 7600X
               ...         
420             인텔 펜티엄 G870
421             인텔 펜티엄 G850
422             인텔 펜티엄 G840
423            인텔 펜티엄 E2200
424          인텔 코어2듀오 E6600
Name: name, Length: 425, dtype: object

In [348]:
for i in range(100):
    if type(req_cpu.loc[i]) == float: continue
    print(f':{i:5d}:{req_cpu.loc[i]:35s}::{cpu.loc[int(id_list[i])-1]}')

:    0:> Intel Pentium III 800 MHz        ::인텔 펜티엄 E2140
:    1:Dual Core CPU                      ::인텔 코어i3-1세대 530
:    2:AMD Athlon 64 X2 Dual Core Processor 4600+, 2.4GHz / Intel Pentium D 805 2.66GHz or higher::인텔 펜티엄 E2140
:    3:AMD Athlon 7850 Dual Core Processor 2.8GHz / Intel Core 2 Duo E7500 2.93 GHz::인텔 코어2듀오 E7500
:    4:Intel Core2 Duo 3.00GHz            ::인텔 코어2듀오 E4300
:    5:1.5 GHz                            ::인텔 펜티엄 E2140
:    6:2.4 GHz                            ::인텔 셀러론 G3950
:    7:Dual Core 1.8 Ghz                  ::인텔 코어i3-1세대 530
:    8:AMD/INTEL DUAL-CORE 2.2 GHZ        ::AMD A4 3300
:    9:Intel Pentium D or AMD Athlon 64 X2::인텔 펜티엄 E2140
:   10:Intel Core 2 Duo E6600 or AMD Athlon 64 X2 5000+ or better::인텔 코어2듀오 E6600
:   11:Intel® Core™ i3                    ::인텔 코어i3-1세대 530
:   12:Intel® Core™ i5-6500 3.2 Ghz       ::인텔 코어i5-1세대 650
:   13:3.0 GHz processor                  ::인텔 펜티엄 E2140
:   14:3.5 GHz processor                  ::인텔 펜티엄 E2140
:   15:1,